## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [15]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
from nltk.corpus import stopwords
from string import punctuation
#print(punctuation)
try:
    sw = stopwords.words("english")
except:
    from nltk import download as nltkdl
    nltkdl('stopwords')
    sw =  stopwords.words("english")


# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [14]:
print(stopwords)

<WordListCorpusReader in '.../corpora/stopwords' (not loaded yet)>


In [3]:
convention_db = sqlite3.connect("2020_Conventions(1).db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes
# <font color='#C7506B'> We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text for each party and prepare it for use in Naive Bayes. </font>

In [4]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            select party, text from conventions
                            ''')
convention_data=[]
for row in query_results :
    #declare variables
    party=row[0]
    text=row[1]
    #clean up text
    text=text.lower()
    text=text.replace('\n',' ')
    for punct in punctuation:
        text=text.replace(punct,'')
    #add words to convention data
    words=text.split()
    convention_dict={'party':party, 'words':words}
    convention_data.append(convention_dict)



Let's look at some random entries and see if they look right. 

In [12]:
random.choices(convention_data,k=10)

[{'party': 'Democratic',
  'words': ['he',
   'would',
   'treat',
   'the',
   'conductor',
   'the',
   'same',
   'as',
   'he',
   'would',
   'the',
   'president',
   'of',
   'the',
   'united',
   'states',
   'that',
   'is',
   'what',
   'dad',
   'taught',
   'us',
   'that',
   'everybody',
   'deserves',
   'to',
   'be',
   'treated',
   'with',
   'dignity',
   'and',
   'respect']},
 {'party': 'Democratic',
  'words': ['for',
   'our',
   'daughters',
   'and',
   'our',
   'granddaughters',
   'today',
   'we',
   'have',
   'broken',
   'the',
   'marble',
   'ceiling']},
 {'party': 'Democratic', 'words': ['caring', 'and', 'principled']},
 {'party': 'Democratic',
  'words': ['tougaloo',
   'college',
   'reflects',
   'the',
   'progression',
   'of',
   'a',
   'people',
   'from',
   'slavery',
   'to',
   'citizenship',
   'to',
   'scholarship',
   'and',
   'leadership',
   'contributing',
   'to',
   'mississippi',
   'and',
   'the',
   'world',
   'our',
   '

# If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [19]:
word_cutoff = 5
#tokens=convention_data[0]['words']
tokens = [w for t, p in convention_data for w in t.split()]
tokens=[]
for row in convention_data:
    tokens+=row['words']

word_dist = nltk.FreqDist(tokens)

feature_words = set()
engstopwords= stopwords.words("english")
for word, count in word_dist.items() :
    if count > word_cutoff and word not in engstopwords:
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")
#print(feature_words)

With a word cutoff of 5, we have 2391 as features in the model.


In [20]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
                   
    """
    ret_dict = dict()
    text_words=text.split()
    for feature_word in fw:
        if feature_word in text_words:
            ret_dict[feature_word]=True
            
    return(ret_dict)

In [21]:
print(conv_features("donald is the president",feature_words))

{'donald': True, 'president': True}


In [ ]:
#looks like I need to filter out stopwords 

In [22]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [ ]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [24]:
#featuresets = []

#for row in convention_data:
 #   party = row["party"]
  #  features = conv_features(row["words"], feature_words)
   # fset = (features, party)
    #featuresets.append(fset)
featuresets = []

for row in convention_data:
    party = row["party"]
    features = conv_features(" ".join(row["words"]), feature_words)
    fset = (features, party)
    featuresets.append(fset)



In [25]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [45]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))
print(dir(classifier))

0.5
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_feature_probdist', '_label_probdist', '_labels', 'classify', 'classify_many', 'labels', 'most_informative_features', 'prob_classify', 'prob_classify_many', 'show_most_informative_features', 'train']


In [27]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

Democrats mention the climate 17.8 times more than Republicans showing that they are more concerned about climate change. Republicans mention religion, defense, and trade more. Democrats talk about votes more. They are more concerned about being elected into office.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [28]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [29]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [40]:
tweet_data = []

for row in results :
    #declare variables
    candidate=row[0]
    party=row[1]
    text=row[2].decode('utf-8')
    #clean up text
    text=text.lower()
    text=text.replace('\n',' ')
    for punct in punctuation:
        text=text.replace(punct,'')
    #add words to convention data
    #words=text.split()
    words=[]
    for word in text.split():
        if word not in engstopwords:
            words.append(word)
   # for sw in stopwords.words("english"):
    #    if sw in words:
     #       words.remove(sw)
    tweet_dict={'party':party, 'words':words,'candidate':candidate}
    tweet_data.append(tweet_dict)

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.
#tweet_data = []

# iterate though database, popualte list
#for row in results:
 #   tweet_dict = {
  #  "candidate" : row[0],
   # "party" : row[1],
    #"words": row[2].split()
    #}
    #
    # tweet_data.append(tweet_dict)
    
word_cutoff = 5
#tokens=convention_data[0]['words']
#tokens = [w for t, p in convention_data for w in t.split()]
tokens=[]
for row in tweet_data:
    tokens+=row['words']

word_dist = nltk.FreqDist(tokens)

feature_words = set()
engstopwords= stopwords.words("english")
for word, count in word_dist.items() :
    if count > word_cutoff:
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")
#print(feature_words)

With a word cutoff of 5, we have 49945 as features in the model.


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [41]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [47]:

#for tweet, party in tweet_data_sample :
for row in tweet_data_sample:
    
    # Fill in the right-hand side above with code that estimates the actual party
    tweet=' '.join(row['words'])
    party= row['party']
    #estimated_party = 'Republican'
    features=conv_features(tweet,feature_words)
    estimated_party= classifier.classify(features)
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let’s make even greater kag 🇺🇸 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1hr c

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [48]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, row in enumerate(tweet_data) :
    tweet=' '.join(row['words'])
    party= row['party']
    features=conv_features(tweet,feature_words)
    estimated_party= classifier.classify(features)
      
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    #estimated_party = "Gotta fill this in"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [49]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3715, 'Democratic': 563}),
             'Democratic': defaultdict(int,
                         {'Republican': 4823, 'Democratic': 901})})

### Reflections
When there was a republican tweet, 3715 were accurately identified as Republican. That was a true positive. 563 were inaccurately identified as Democratic. Those were false negatives. The accuracy is 86%. 
When there was a democratic tweet 4823 were accurately identified. When there was a democratic tweet 901 were accurately identified. The accuracy is 84%. 
Generally speaking, the classfier was correct 85.38% of the time.

In [50]:
4823/(4823+901)

0.8425925925925926